In [1]:
import pandas as pd
import numpy as np
import json 
import re
from geojson import Feature, FeatureCollection, MultiPoint, Point
import uuid

In [2]:
class Vehicle:
    def __init__(
        self,
        id,
        department,
        make,
        model,
        year,
        vehicle_class
        ):
        self.id = id
        self.department = department
        self.make = make
        self.model = model
        self.year = year
        self.vehicle_class = vehicle_class

In [3]:
class Trip:
    def __init__(
        self,
        vehicle,
        start_time,
        end_time,
        date,
        ):
        self.id = str(uuid.uuid4())
        self.vehicle = vehicle
        self.start_time = start_time
        self.end_time = end_time
        self.date = date
        self.stops = []
        self.num_stops = 0
        self.distance = 0
        self.total_duration = 0
        self.idle_duration = 0
        self.driving_duration = 0
        self.stopped_duration = 0
        self.geojson = []
        self.show = True

    def add_stop(self, stop):
        #self.stops.append(stop.__dict__)
        stop_properties = {
            'vehicle': self.vehicle,
            'stopDuration': stop.stop_duration,
            'idleDuration': stop.idle_duration,
            'startTime': stop.start_time,
            'endTime': stop.end_time,
            'drivingDuration': stop.driving_duration,
            'distance': stop.distance,
            'stopNum': self.num_stops
        }
        point = Point(stop.location)
        feature = Feature(geometry=point, properties=stop_properties)
        self.geojson.append(feature)
        
        self.num_stops += 1
        self.distance += stop.distance
        self.total_duration += (stop.stop_duration + stop.driving_duration)
        self.idle_duration += stop.idle_duration
        self.driving_duration += stop.driving_duration
        self.stopped_duration += stop.stop_duration

    def calculate_geojson(self):
        coords = []
        for stop in self.stops:
            coords.append(stop['location'])
        self.geojson = Feature(geometry=MultiPoint(coords))



In [4]:
class Department:
    def __init__ (
        self,
        name,
        vehicles
    ):
        self.name = name
        self.num_vehicles = len(vehicles);
        self.vehicles = vehicles


In [5]:
class Stop:
    def __init__(
        self,
        location_lat,
        location_long,
        stop_duration,
        idle_duration,
        driving_duration,
        start_time,
        end_time,
        distance
        ):
        self.location = (location_long, location_lat)
        self.stop_duration = stop_duration
        self.idle_duration = idle_duration
        self.driving_duration = driving_duration
        self.start_time = start_time
        self.end_time = end_time
        self.distance = distance

In [6]:
fleet_df = pd.read_csv('../src/data/vehicle_details/FleetDetails.csv')

In [7]:
fleet_df = fleet_df.fillna('')

In [8]:
fleet_df['VehicleName'] = [str(name).split(' ')[0] for name in fleet_df['VehicleName']]
fleet_df['VehicleName'] = [re.sub('[\W_]+', '', str(name)) for name in fleet_df['VehicleName']]

In [9]:
fleet_obj_arr = []
for idx, row in fleet_df.iterrows():
    vehicle = Vehicle(row['VehicleName'], row['Department'], row['Make'], row['Model'], row['Year'], row['Class'])
    fleet_obj_arr.append(vehicle.__dict__)

In [10]:
dept_obj_arr = []
groups = fleet_df.groupby('Department')
for name, rows in groups:
    dept_vehicles = []
    for idx, row in rows.iterrows():
        dept_vehicles.append(row['VehicleName'])
    dept_obj_arr.append(Department(name, dept_vehicles).__dict__)

In [11]:
trips_df = pd.read_csv('9-27-9-29_trip_latlng.csv')
trips_df = trips_df.rename(columns={'Unnamed: 0': 'idx'})
trips_df['trip_num'] = 0
trips_df['StopDuration'] = trips_df['StopDuration'].apply(lambda x: int(x[0]) * 60 + int(x[-2:]))

In [12]:
grouped = trips_df.groupby('Device')
trip_num = 0
for name, group in grouped:
    last_row = group.iloc[0]
    for i, row in group.iterrows():
        if row.idx - last_row.idx > 1:
            trip_num += 1
        trips_df['trip_num'].iloc[row.idx] = trip_num
        last_row = row
    trip_num += 1

C:\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
trips = []
grouped = trips_df.groupby('trip_num')
for name, group in grouped:
    trip = Trip(group.iloc[0]['Device'], group.iloc[0]['StartDate'], group.iloc[-1]['EndDate'], group.iloc[0]['StartDate'])
    for i, row in group.iterrows():
        stop = Stop(row['lat'], row['long'], row['StopDuration'], row['IdleDuration'], row['Duration'], row['StartDate'], row['EndDate'], row['Distance'])
        trip.add_stop(stop)
    trips.append(trip.__dict__)



In [14]:
fleet_details_obj = {'vehicles': fleet_obj_arr}
trip_details_obj = {'trips': trips}
dept_details_obj = {'departments': dept_obj_arr}

In [15]:
with open("fleet_details.json", "w") as outfile:
    json.dump(fleet_details_obj, outfile)

with open("trips_9-27_9-29.json", "w") as outfile:
    json.dump(trip_details_obj, outfile)

    
with open("depts_9-27_9-29.json", "w") as outfile:
    json.dump(dept_details_obj, outfile)